# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 10:05AM,248536,10,MSP214828,"Methapharm, Inc.",Released
1,Oct 13 2022 10:04AM,248535,10,MSP214836,Methapharm-G,Released
2,Oct 13 2022 9:55AM,248533,10,MSP214837,"Methapharm, Inc.",Released
3,Oct 13 2022 9:47AM,248532,19,N17802,"NAPP Technologies, LLC",Released
4,Oct 13 2022 9:46AM,248531,10,0086074455,ISDIN Corporation,Released
5,Oct 13 2022 9:46AM,248531,10,0086074454,ISDIN Corporation,Released
6,Oct 13 2022 9:46AM,248531,10,0086074462,ISDIN Corporation,Released
7,Oct 13 2022 9:46AM,248531,10,0086074459,ISDIN Corporation,Released
8,Oct 13 2022 9:46AM,248531,10,0086074458,ISDIN Corporation,Released
9,Oct 13 2022 9:46AM,248531,10,0086074461,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,248531,Released,26
21,248532,Released,1
22,248533,Released,1
23,248535,Released,1
24,248536,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248531,NaN,26.0
248532,NaN,1.0
248533,NaN,1.0
248535,NaN,1.0
248536,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248081,1.0,0.0
248083,1.0,0.0
248087,0.0,1.0
248089,0.0,1.0
248091,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248081,1,0
248083,1,0
248087,0,1
248089,0,1
248091,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248081,1,0
1,248083,1,0
2,248087,0,1
3,248089,0,1
4,248091,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248081,1,
1,248083,1,
2,248087,,1
3,248089,,1
4,248091,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc."
1,Oct 13 2022 10:04AM,248535,10,Methapharm-G
2,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc."
3,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC"
4,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation
30,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation
79,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation
94,Oct 13 2022 8:42AM,248526,19,ACG North America LLC
96,Oct 13 2022 8:23AM,248525,19,ACG North America LLC
97,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",,1
1,Oct 13 2022 10:04AM,248535,10,Methapharm-G,,1
2,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc.",,1
3,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",,1
4,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation,,26
5,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,,49
6,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,,15
7,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,,2
8,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,,1
9,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1,
1,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1,
2,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc.",1,
3,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",1,
4,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation,26,
5,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,49,
6,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,15,
7,Oct 13 2022 8:42AM,248526,19,ACG North America LLC,2,
8,Oct 13 2022 8:23AM,248525,19,ACG North America LLC,1,
9,Oct 12 2022 4:56PM,248521,21,"NBTY Global, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1,
1,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1,
2,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc.",1,
3,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",1,
4,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation,26,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1.0,NaN
1,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1.0,NaN
2,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc.",1.0,NaN
3,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",1.0,NaN
4,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation,26.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1.0,0.0
1,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1.0,0.0
2,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc.",1.0,0.0
3,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",1.0,0.0
4,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation,26.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2733533,116.0,4.0
18,248462,1.0,0.0
19,994047,5.0,1.0
21,1488952,3.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2733533,116.0,4.0
1,18,248462,1.0,0.0
2,19,994047,5.0,1.0
3,21,1488952,3.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,116.0,4.0
1,18,1.0,0.0
2,19,5.0,1.0
3,21,3.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,116.0
1,18,Released,1.0
2,19,Released,5.0
3,21,Released,3.0
4,10,Executing,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,18,19,21
Status,,,,
Executing,4.0,0.0,1.0,3.0
Released,116.0,1.0,5.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,18,19,21
0,Executing,4.0,0.0,1.0,3.0
1,Released,116.0,1.0,5.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,18,19,21
0,Executing,4.0,0.0,1.0,3.0
1,Released,116.0,1.0,5.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()